# PySpark Temporary and Global Temporary Views

This notebook explains how to use `createOrReplaceTempView` and `createGlobalTempView` in PySpark with theory, examples, and practice exercises.

## Temporary Views in PySpark

**Definition**: A temporary view is a table-like structure created from a DataFrame that is registered to the current SparkSession. It can be queried using SQL.

- Scope: Session-local
- Use SQL directly on top of DataFrames
- Not persisted to disk

**Syntax**:
```python
df.createOrReplaceTempView("view_name")
spark.sql("SELECT * FROM view_name").show()
```

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TempTableExample").getOrCreate()

data = [("Alice", 34), ("Bob", 45), ("Cathy", 29)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)

df.createOrReplaceTempView("people")
spark.sql("SELECT * FROM people WHERE Age > 30").show()

+-----+---+
| Name|Age|
+-----+---+
|Alice| 34|
|  Bob| 45|
+-----+---+



## Global Temporary Views

**Definition**: A global temporary view is shared across all SparkSessions in the same Spark application. It is registered in the `global_temp` database.

- Scope: All sessions in same app
- Use SQL with `global_temp.view_name`
- Not persisted to disk

**Syntax**:
```python
df.createGlobalTempView("view_name")
spark.sql("SELECT * FROM global_temp.view_name").show()
```

Note: You must prefix it with global_temp. when querying.

## Exercise 1: Uppercase Names from Temp View
**Task**: Create a temp view and select names in uppercase where age < 35

In [2]:
data = [("Anna", 30), ("John", 25), ("Laura", 45)]
df = spark.createDataFrame(data, ["Name", "Age"])
df.createOrReplaceTempView("test_view")

spark.sql("SELECT UPPER(Name) AS Name_Upper FROM test_view WHERE Age < 35").show()

+----------+
|Name_Upper|
+----------+
|      ANNA|
|      JOHN|
+----------+



## Exercise 2: Join Two Temp Views
**Task**: Join a department view with a people view and select combined data.

In [3]:
departments = [("Anna", "HR"), ("John", "IT")]
df1 = spark.createDataFrame(departments, ["Name", "Dept"])
df1.createOrReplaceTempView("dept_view")

people = [("Anna", 30), ("John", 25)]
df2 = spark.createDataFrame(people, ["Name", "Age"])
df2.createOrReplaceTempView("people_view")

spark.sql("""
    SELECT p.Name, p.Age, d.Dept
    FROM people_view p
    JOIN dept_view d ON p.Name = d.Name
""").show()

+----+---+----+
|Name|Age|Dept|
+----+---+----+
|Anna| 30|  HR|
|John| 25|  IT|
+----+---+----+



## Exercise 3: SQL Functions in a Temp View
**Task**: Use UPPER() and calculate birth year assuming current year is 2025

In [4]:
# Simulating CSV content
data = [("Alice", 34), ("Bob", 45), ("Cathy", 29)]
df = spark.createDataFrame(data, ["Name", "Age"])
df.createOrReplaceTempView("people")

spark.sql("""
    SELECT 
        Name,
        UPPER(Name) AS Name_Upper,
        Age,
        2025 - Age AS BirthYear
    FROM people
    WHERE Age IS NOT NULL
""").show()

+-----+----------+---+---------+
| Name|Name_Upper|Age|BirthYear|
+-----+----------+---+---------+
|Alice|     ALICE| 34|     1991|
|  Bob|       BOB| 45|     1980|
|Cathy|     CATHY| 29|     1996|
+-----+----------+---+---------+

